In [6]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os

In [1]:
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat

--2019-01-23 10:42:39--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3903344 (3.7M)
Saving to: ‘97.mat’

100%[======================================>] 3,903,344   4.72MB/s   in 0.8s   

2019-01-23 10:42:40 (4.72 MB/s) - ‘97.mat’ saved [3903344/3903344]

--2019-01-23 10:42:40--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7742720 (7.4M)
Saving to: ‘98.mat’

100%[======================================>] 7,742,720   6.50MB/s   in 1.1s   

2019-01-23 10:42:42 (6.50 MB/s) - ‘98.mat’ saved [7742720/7742

In [2]:
!mkdir cwr_healthy

mkdir: cannot create directory ‘cwr_healthy’: File exists


In [7]:
!mv *.mat cwr_healthy/

In [3]:
!ls -lahr cwr_healthy

total 100M
-rw-------.  1 s6f6-9d8798f6e89df9-29ae4b34f4d2 users  15M Sep  7  2011 99.mat
-rw-------.  1 s6f6-9d8798f6e89df9-29ae4b34f4d2 users 7.4M Sep  7  2011 98.mat
-rw-------.  1 s6f6-9d8798f6e89df9-29ae4b34f4d2 users 3.8M Sep  7  2011 97.mat
-rw-------.  1 s6f6-9d8798f6e89df9-29ae4b34f4d2 users 7.5M Sep  7  2011 100.mat
drwx------. 12 s6f6-9d8798f6e89df9-29ae4b34f4d2 users 4.0K Jan 23 10:42 ..
drwx------.  2 s6f6-9d8798f6e89df9-29ae4b34f4d2 users 4.0K Jan 22 10:56 .


In [7]:
nlf = sio.loadmat('./cwr_healthy/100.mat')
nlf

{'X100RPM': array([[1725]], dtype=uint16),
 'X100_DE_time': array([[ 0.01460308],
        [ 0.05444862],
        [ 0.10764554],
        ..., 
        [-0.02357354],
        [ 0.00521538],
        [ 0.04777292]]),
 'X100_FE_time': array([[ 0.19292182],
        [ 0.16436364],
        [ 0.09081091],
        ..., 
        [ 0.10930182],
        [ 0.05218545],
        [-0.00452   ]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Fri Jan 28 11:25:48 2000',
 '__version__': '1.0'}

In [8]:
type(nlf['X100_DE_time'])

numpy.ndarray

In [16]:
def read_folder(folder):
    data = 'dummy'
    skip = False
    for file in os.listdir(folder):
        file_id = file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                    data
                except NameError:
                    data = a
                else:
                    if (data.shape[0] != a.shape[0]):
                        print('skipping ' + file_id)
                        skip = True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip=False
            continue
        id = np.repeat(file_id,data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id,data))
        if data.shape[1] == 2:
            zeros = np.repeat(float(0),data.shape[0])
            zeros.shape =(data.shape[0],1)
            data = np.hstack((data,zeros))
        try:
            result
        except NameError:
            result = data
        else:
            result = np.vstack((result,data))
    return result

In [17]:
result_healthy = read_folder('./cwr_healthy/')

skipping 99
skipping 99


In [23]:
result_healthy

array([['98', '0.023216363636363632', '0.046104'],
       ['98', '0.08115454545454545', '-0.03713353846153846'],
       ['98', '0.09533090909090909', '-0.08949599999999999'],
       ..., 
       ['100', '0.10930181818181818', '-0.02357353846153846'],
       ['100', '0.05218545454545454', '0.005215384615384615'],
       ['100', '-0.00452', '0.047772923076923074']],
      dtype='<U32')

In [24]:
pdf = pd.DataFrame(result_healthy)

In [25]:
pdf.to_csv('result_healthy_pandas.csv', header=False, index=True)

In [26]:
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/48k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-fan-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!mkdir cwr_faulty
!mv *.mat cwr_faulty/

--2019-01-23 10:56:48--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2910768 (2.8M)
Saving to: ‘105.mat’

100%[======================================>] 2,910,768   4.05MB/s   in 0.7s   

2019-01-23 10:56:50 (4.05 MB/s) - ‘105.mat’ saved [2910768/2910768]

--2019-01-23 10:56:50--  http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/118.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942112 (2.8M)
Saving to: ‘118.mat’

100%[======================================>] 2,942,112   1.33MB/s   in 2.1s   

2019-01-23 10:56:52 (1.33 MB/s) - ‘118.mat’ saved [294211

In [28]:
!ls cwr_faulty/

100.mat  135.mat  170.mat  203.mat  237.mat  270.mat  291.mat	306.mat
105.mat  136.mat  171.mat  204.mat  238.mat  271.mat  292.mat	307.mat
106.mat  137.mat  172.mat  209.mat  239.mat  272.mat  293.mat	309.mat
107.mat  138.mat  174.mat  210.mat  240.mat  273.mat  294.mat	310.mat
108.mat  144.mat  175.mat  211.mat  241.mat  274.mat  295.mat	311.mat
109.mat  145.mat  176.mat  212.mat  246.mat  275.mat  296.mat	312.mat
110.mat  146.mat  177.mat  213.mat  247.mat  276.mat  297.mat	313.mat
111.mat  147.mat  185.mat  214.mat  248.mat  277.mat  298.mat	315.mat
112.mat  148.mat  186.mat  215.mat  249.mat  278.mat  299.mat	316.mat
118.mat  149.mat  187.mat  217.mat  250.mat  279.mat  3001.mat	317.mat
119.mat  150.mat  188.mat  222.mat  251.mat  280.mat  3002.mat	318.mat
120.mat  151.mat  189.mat  223.mat  252.mat  281.mat  3003.mat	97.mat
121.mat  156.mat  190.mat  224.mat  253.mat  282.mat  3004.mat	98.mat
122.mat  158.mat  191.mat  225.mat  258.mat  283.mat  3005.mat	99.mat
123.mat  159.mat  

In [31]:
result_faulty = read_folder('./cwr_faulty/')

skipping 175
skipping 99
skipping 99


MemoryError: 

In [ ]:
pdf = pd.DataFrame(result_faulty)

In [ ]:
pdf.to_csv('result_faulty_pandas.csv', header=False, index=True)

In [ ]:
df_healhty = spark.read.csv('result_healthy_pandas.csv')
df_healhty.createOrReplaceTempView('df_healhty')

In [ ]:
spark.sql('select _c1,count(_c1) as cn from df_healhty group by _c1 order by cn asc').show()

In [ ]:
df_faulty = spark.read.csv('result_faulty_pandas.csv')
df_faulty.createOrReplaceTempView('df_faulty')

In [ ]:
spark.sql('select _c1,count(_c1) as cn from df_faulty group by _c1 order by cn asc').show()

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
df_healhty.write.parquet(bmos.url('courseraai', 'cwr_healthy1.parquet'))

In [ ]:
df_faulty.write.parquet(bmos.url('courseraai', 'cwr_faulty1.parquet'))

In [ ]:
df_faulty.count()